In [1]:
Nusers = 1000
Nmovies = 100

In [2]:
import math
import numpy as np
import random
# S = np.random.randint(2, 6)   # Number of satellites (between 2 to 5)
# U = np.random.randint(5, 15)  # Number of UAVs (between 5 to 15)
# K = np.random.randint(2, 6)   # Number of CSPs (between 2 to 5)
# F = np.random.randint(10, 50) # Number of content files (10 to 50)
# N = np.random.randint(20, 100) # Number of GUs (20 to 100)
# # Maximum number of files UAV can store (F' << F)
# F_prime = np.random.randint(1, max(2, F // 2))  # At most 1/2 of total files
S = 2
U = 8
K = 2
N = Nusers
F = Nmovies
F_prime = (7*F)//16
M_u = 20
L = np.full(U, (2*F_prime)//3) # define transmissino load for each uav
# print(L)

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load datasets
movies = pd.read_csv("/content/movies.dat", sep="::",nrows=Nmovies, names=['movieId', 'title', 'genres'], engine='python', encoding='ISO-8859-1')
ratings = pd.read_csv("/content/ratings.dat", sep="::", names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')
users = pd.read_csv("/content/users.dat", sep="::",nrows=Nusers, names=['userId', 'gender', 'age', 'occupation', 'zip-code'], engine='python')

# Merge datasets
merged_data = pd.merge(ratings, users, on='userId', how='inner')
merged_data = pd.merge(merged_data, movies, on='movieId', how='inner')


In [4]:
# Sorting by timestamp
merged_data = merged_data.sort_values(by='timestamp')

# Define number of timeslots
num_timeslots = 10

# Create timeslot labels
merged_data['timeslot'] = pd.qcut(merged_data['timestamp'], q=num_timeslots, labels=[int(f'{i+1}') for i in range(num_timeslots)])

# Display timeslot distribution
print(merged_data['timeslot'].value_counts())


merged_data


timeslot
9     515
1     514
2     514
3     514
5     514
4     514
8     514
7     514
6     513
10    513
Name: count, dtype: int64


,userId,movieId,rating,timestamp,gender,age,occupation,zip-code,title,genres,timeslot
5138,1000,1,5,975041651,F,25,6,90027,Toy Story (1995),Animation|Children's|Comedy,1
5117,999,6,2,975044156,M,25,15,62558,Heat (1995),Action|Crime|Thriller,1
5123,999,21,2,975044156,M,25,15,62558,Get Shorty (1995),Action|Comedy|Drama,1
5132,999,70,2,975045753,M,25,15,62558,From Dusk Till Dawn (1996),Action|Comedy|Crime|Horror|Thriller,1
5137,999,95,2,975045995,M,25,15,62558,Broken Arrow (1996),Action|Thriller,1
...,...,...,...,...,...,...,...,...,...,...,...
3792,736,1,3,1045710680,M,18,12,07070,Toy Story (1995),Animation|Children's|Comedy,10
3796,736,32,5,1045711057,M,18,12,07070,Twelve Monkeys (1995),Drama|Sci-Fi,10
3797,736,47,5,1045711096,M,18,12,07070,Seven (Se7en) (1995),Crime|Thriller,10
3793,736,6,4,1045711853,M,18,12,07070,Heat (1995),Action|Crime|Thriller,10


In [5]:
merged_data.drop("title",axis=1,inplace=True)

In [6]:
merged_data

,userId,movieId,rating,timestamp,gender,age,occupation,zip-code,genres,timeslot
5138,1000,1,5,975041651,F,25,6,90027,Animation|Children's|Comedy,1
5117,999,6,2,975044156,M,25,15,62558,Action|Crime|Thriller,1
5123,999,21,2,975044156,M,25,15,62558,Action|Comedy|Drama,1
5132,999,70,2,975045753,M,25,15,62558,Action|Comedy|Crime|Horror|Thriller,1
5137,999,95,2,975045995,M,25,15,62558,Action|Thriller,1
...,...,...,...,...,...,...,...,...,...,...
3792,736,1,3,1045710680,M,18,12,07070,Animation|Children's|Comedy,10
3796,736,32,5,1045711057,M,18,12,07070,Drama|Sci-Fi,10
3797,736,47,5,1045711096,M,18,12,07070,Crime|Thriller,10
3793,736,6,4,1045711853,M,18,12,07070,Action|Crime|Thriller,10


First find all unique genres list

In [7]:
# Get unique genres from the dataset
unique_genres = set()

# Iterate through all genres and split them by '|'
for genre_list in merged_data['genres'].dropna():
    unique_genres.update(genre_list.split('|'))

# Convert set to a sorted list
unique_genres = sorted(unique_genres)

# Display unique genres
print(len(unique_genres)," : ",unique_genres)


16  :  ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War']


In [8]:
for genre in unique_genres:
    merged_data[genre] = merged_data['genres'].apply(lambda x: 1 if genre in x.split('|') else 0)


merged_data.drop(columns=['genres'], inplace=True)


In [9]:
merged_data

,userId,movieId,rating,timestamp,gender,age,occupation,zip-code,timeslot,Action,...,Documentary,Drama,Fantasy,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War
5138,1000,1,5,975041651,F,25,6,90027,1,0,...,0,0,0,0,0,0,0,0,0,0
5117,999,6,2,975044156,M,25,15,62558,1,1,...,0,0,0,0,0,0,0,0,1,0
5123,999,21,2,975044156,M,25,15,62558,1,1,...,0,1,0,0,0,0,0,0,0,0
5132,999,70,2,975045753,M,25,15,62558,1,1,...,0,0,0,1,0,0,0,0,1,0
5137,999,95,2,975045995,M,25,15,62558,1,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792,736,1,3,1045710680,M,18,12,07070,10,0,...,0,0,0,0,0,0,0,0,0,0
3796,736,32,5,1045711057,M,18,12,07070,10,0,...,0,1,0,0,0,0,0,1,0,0
3797,736,47,5,1045711096,M,18,12,07070,10,0,...,0,0,0,0,0,0,0,0,1,0
3793,736,6,4,1045711853,M,18,12,07070,10,1,...,0,0,0,0,0,0,0,0,1,0


In [10]:
merged_data
merged_data['gender'] = merged_data['gender'].map({'M': 0,'F':1})

In [11]:
merged_data.describe()
zip_mapping = {zip_code: idx for idx, zip_code in enumerate(merged_data["zip-code"].unique())}

# Apply mapping
merged_data['zip-code'] =merged_data['zip-code'].map(zip_mapping)

In [12]:
df = merged_data[['userId', 'gender', 'age', 'occupation', 'zip-code',
       'movieId'] + unique_genres+ ['rating']]

In [13]:
dataset_test = users.merge(movies, how='cross')
for genre in unique_genres:
    dataset_test[genre] = dataset_test['genres'].apply(lambda x: 1 if genre in x.split('|') else 0)


dataset_test.drop(columns=['genres'], inplace=True)
dataset_test.drop(columns=['title'], inplace=True)
dataset_test['gender'] = dataset_test['gender'].map({'M': 0,'F':1})

zip_mapping = {zip_code: idx for idx, zip_code in enumerate(dataset_test["zip-code"].unique())}

# Apply mapping
dataset_test['zip-code'] = dataset_test['zip-code'].map(zip_mapping)
print(dataset_test)
dataset_test = dataset_test.merge(df[['userId', 'movieId', 'rating']],
                                  on=['userId', 'movieId'],
                                  how='left')
dataset_test.fillna(0,inplace=True)


# Unique users and movies
unique_users = dataset_test["userId"].unique()      # = Nusers
unique_movies = dataset_test["movieId"].unique()        # = Nmovies

# Create mapping from original IDs to indices (starting from 1)
user_to_index = {user: idx + 1 for idx, user in enumerate(unique_users)}
movie_to_index = {movie: idx + 1 for idx, movie in enumerate(unique_movies)}

# Replace values in the DataFrame
dataset_test["userId"] = dataset_test["userId"].map(user_to_index)
dataset_test["movieId"] = dataset_test["movieId"].map(movie_to_index)



dataset_test.to_csv('dataset_test.csv')

       userId  gender  age  occupation  zip-code  movieId  Action  Adventure  \
0           1       1    1          10         0        1       0          0   
1           1       1    1          10         0        2       0          1   
2           1       1    1          10         0        3       0          0   
3           1       1    1          10         0        4       0          0   
4           1       1    1          10         0        5       0          0   
...       ...     ...  ...         ...       ...      ...     ...        ...   
99995    1000       1   25           6       269       97       0          0   
99996    1000       1   25           6       269       98       1          0   
99997    1000       1   25           6       269       99       0          0   
99998    1000       1   25           6       269      100       0          0   
99999    1000       1   25           6       269      101       0          0   

       Animation  Children's  ...  Docu

In [14]:
dataset_test = pd.read_csv('dataset_test.csv')


In [15]:
dataset_test

,Unnamed: 0,userId,gender,age,occupation,zip-code,movieId,Action,Adventure,Animation,...,Drama,Fantasy,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,rating
0,0,1,1,1,10,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,5.0
1,1,1,1,1,10,0,2,0,1,0,...,0,1,0,0,0,0,0,0,0,0.0
2,2,1,1,1,10,0,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0.0
3,3,1,1,1,10,0,4,0,0,0,...,1,0,0,0,0,0,0,0,0,0.0
4,4,1,1,1,10,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1000,1,25,6,269,96,0,0,0,...,1,0,0,0,0,0,0,0,0,0.0
99996,99996,1000,1,25,6,269,97,1,0,0,...,0,0,0,0,0,0,0,1,0,0.0
99997,99997,1000,1,25,6,269,98,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
99998,99998,1000,1,25,6,269,99,0,0,0,...,1,0,0,0,0,0,0,1,0,0.0


In [16]:
dataset_test

,Unnamed: 0,userId,gender,age,occupation,zip-code,movieId,Action,Adventure,Animation,...,Drama,Fantasy,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,rating
0,0,1,1,1,10,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,5.0
1,1,1,1,1,10,0,2,0,1,0,...,0,1,0,0,0,0,0,0,0,0.0
2,2,1,1,1,10,0,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0.0
3,3,1,1,1,10,0,4,0,0,0,...,1,0,0,0,0,0,0,0,0,0.0
4,4,1,1,1,10,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1000,1,25,6,269,96,0,0,0,...,1,0,0,0,0,0,0,0,0,0.0
99996,99996,1000,1,25,6,269,97,1,0,0,...,0,0,0,0,0,0,0,1,0,0.0
99997,99997,1000,1,25,6,269,98,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
99998,99998,1000,1,25,6,269,99,0,0,0,...,1,0,0,0,0,0,0,1,0,0.0


In [17]:
df

,userId,gender,age,occupation,zip-code,movieId,Action,Adventure,Animation,Children's,...,Drama,Fantasy,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,rating
5138,1000,1,25,6,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,5
5117,999,0,25,15,1,6,1,0,0,0,...,0,0,0,0,0,0,0,1,0,2
5123,999,0,25,15,1,21,1,0,0,0,...,1,0,0,0,0,0,0,0,0,2
5132,999,0,25,15,1,70,1,0,0,0,...,0,0,1,0,0,0,0,1,0,2
5137,999,0,25,15,1,95,1,0,0,0,...,0,0,0,0,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792,736,0,18,12,183,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,3
3796,736,0,18,12,183,32,0,0,0,0,...,1,0,0,0,0,0,1,0,0,5
3797,736,0,18,12,183,47,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5
3793,736,0,18,12,183,6,1,0,0,0,...,0,0,0,0,0,0,0,1,0,4


In [18]:
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [19]:
X = df[['userId', 'gender', 'age', 'occupation', 'zip-code',
       'movieId',] + unique_genres]
y = df['rating']



X_train, y_train = X, y


In [20]:
X_train

,userId,gender,age,occupation,zip-code,movieId,Action,Adventure,Animation,Children's,...,Documentary,Drama,Fantasy,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War
5138,1000,1,25,6,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5117,999,0,25,15,1,6,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5123,999,0,25,15,1,21,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5132,999,0,25,15,1,70,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
5137,999,0,25,15,1,95,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792,736,0,18,12,183,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3796,736,0,18,12,183,32,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3797,736,0,18,12,183,47,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3793,736,0,18,12,183,6,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
# Train data
X_train_raw = X
y_train = y

# Normalize
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
# X_test_scaled = scaler.transform(X_test_raw)

# Reshape for LSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
# X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [22]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

# Define the model
model = Sequential()

# Layer 1: First LSTM layer with 24 units and return_sequences=True
model.add(LSTM(24, return_sequences=True, input_shape=(1, X_train_scaled.shape[1])))

# Layer 2: Second LSTM layer with 24 units
model.add(LSTM(24, return_sequences=True))

# Layer 3: Third LSTM layer with 12 units
model.add(LSTM(12))

# Output layer: Fully connected (Dense) layer with linear activation
model.add(Dense(1, activation='linear'))

# Compile the model with Adam optimizer
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mean_absolute_error')
model.fit(X_train_lstm, y_train, epochs=20, batch_size=32, verbose=1)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


161/161 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 3.4942
Epoch 2/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.2488
Epoch 3/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8702
Epoch 4/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8354
Epoch 5/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8490
Epoch 6/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8408
Epoch 7/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8345
Epoch 8/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8490
Epoch 9/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8539
Epoch 10/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8302
Epoch 11/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8364
Epoch 12/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8222
Epoch 13/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8268
Epoch 14/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8136
Epoch 15/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8192


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 24)          │         4,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1, 24)          │         4,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 12)             │         1,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,017 (128.98 KB)

 Trainable params: 11,005 (42.99 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,012 (85.99 KB)

In [24]:
X_test = dataset_test[['userId', 'gender', 'age', 'occupation', 'zip-code',
       'movieId']+ unique_genres]
y_test = dataset_test['rating']

In [25]:
X_test_scaled = scaler.transform(X_test)  # Use .transform, not .fit_transform
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))
preds = model.predict(X_test_lstm)


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


In [26]:
actual_q_i_f =  dataset_test.pivot_table(index='userId', columns='movieId', values='rating', fill_value=0)
actual_q_i_f


predicted_q_i_f = np.zeros((Nusers, Nmovies))


unique_users = dataset_test["userId"].unique()      # = Nusers
unique_movies = dataset_test["movieId"].unique()        # = Nmovies


test_user_ids = dataset_test["userId"].values
test_movie_ids = dataset_test["movieId"].values
print(len(pd.Series(test_user_ids).unique()))

user_to_index = {user: idx for idx, user in enumerate(unique_users)}
movie_to_index = {movie: idx for idx, movie in enumerate(unique_movies)}

# Fill the matrix
for i in range(len(preds)):
    user_idx = user_to_index[test_user_ids[i]]
    movie_idx = movie_to_index[test_movie_ids[i]]
    # print(user_idx,movie_idx)
    predicted_q_i_f[user_idx][movie_idx] = preds[i]
    # print(predicted_q_i_f[user_idx][movie_idx])

print("Matrix filled with predicted ratings.")
# Resulting 2D array: predicted_q_i_f
print(predicted_q_i_f.shape)


# if np.all(predicted_q_i_f == 0):
#     print("The matrix contains only zeros.")
# else:
#     print("The matrix has non-zero values.")

non_zero_count = np.count_nonzero(predicted_q_i_f)
total_elements = predicted_q_i_f.size

print(f"Non-zero elements: {non_zero_count}")
print(f"Total elements: {total_elements}")
print(f"Percentage of non-zero values: {(non_zero_count / total_elements) * 100:.4f}%")



1000
Matrix filled with predicted ratings.
(1000, 100)
Non-zero elements: 100000
Total elements: 100000
Percentage of non-zero values: 100.0000%


<ipython-input-26-8b172ab6d75e>:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_q_i_f[user_idx][movie_idx] = preds[i]


In [27]:
actual_q_i_f

movieId,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
predicted_q_i_f = pd.DataFrame(predicted_q_i_f)
predicted_q_i_f.index = range(1, predicted_q_i_f.shape[0] + 1)
predicted_q_i_f.columns = range(1, predicted_q_i_f.shape[1] + 1)
predicted_q_i_f.index.name = "userId"
predicted_q_i_f.columns.name = "movieId"
predicted_q_i_f

movieId,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
userId,,,,,,,,,,,,,,,,,,,,,
1,3.917006,2.098409,2.566328,3.834065,2.159822,3.881517,2.578952,2.905046,1.269738,3.000258,...,3.909653,2.756325,3.801654,2.134922,2.442323,3.801910,2.130472,2.909997,3.909372,2.451720
2,3.970221,2.499382,2.852076,3.900133,2.513787,3.944236,2.864269,3.249640,1.591116,3.254276,...,3.962074,3.034430,3.887503,2.491408,2.788401,3.887647,2.486847,3.235046,3.961802,2.797312
3,3.925278,1.985509,2.396349,3.824212,1.993853,3.894605,2.412149,2.873064,1.167539,2.980483,...,3.920786,2.650623,3.815476,2.103864,2.344251,3.815901,2.099504,2.915356,3.920716,2.357098
4,3.957449,2.297999,2.720465,3.879702,2.346438,3.929350,2.734748,3.115590,1.396154,3.166700,...,3.951598,2.938618,3.865326,2.381745,2.666841,3.865569,2.377913,3.140913,3.951378,2.677759
5,3.924375,2.004577,2.389456,3.821746,1.993447,3.894229,2.404640,2.885782,1.190289,2.981708,...,3.918913,2.634449,3.815182,2.093377,2.331454,3.815598,2.088614,2.910607,3.918852,2.343827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,4.049096,3.603610,3.706110,4.023660,3.581392,4.037981,3.709926,3.853724,3.092742,3.826192,...,4.045955,3.761090,4.021854,3.549235,3.680110,4.021897,3.546925,3.835489,4.045848,3.683062
997,4.036267,3.440118,3.580516,4.003202,3.408360,4.023592,3.585758,3.773049,2.825080,3.756188,...,4.033862,3.659041,4.003686,3.409188,3.547571,4.003798,3.406492,3.758538,4.033813,3.552090
998,4.039259,3.512207,3.604935,4.009305,3.452715,4.025564,3.609674,3.804489,2.870088,3.753814,...,4.033790,3.672760,4.005332,3.383349,3.568361,4.005334,3.380478,3.765417,4.033587,3.571718


In [29]:
print("Actual\t|Predicted\t|Difference\t|")
for i in range(len(y_test)):  # Use len(y_test) instead of a fixed number
    print(y_test.iloc[i], "\t|", preds[i], "\t|", y_test.iloc[i] - preds[i], "\t|")


Streaming output truncated to the last 5000 lines.
5.0 	| [4.0559] 	| [0.9440999] 	|
0.0 	| [3.6718636] 	| [-3.67186356] 	|
0.0 	| [3.7744083] 	| [-3.77440834] 	|
0.0 	| [4.034879] 	| [-4.03487921] 	|
0.0 	| [3.6734853] 	| [-3.67348528] 	|
0.0 	| [4.0454183] 	| [-4.04541826] 	|
0.0 	| [3.7775679] 	| [-3.77756786] 	|
0.0 	| [3.887659] 	| [-3.88765907] 	|
0.0 	| [3.203998] 	| [-3.20399809] 	|
0.0 	| [3.8582664] 	| [-3.85826635] 	|
0.0 	| [4.0513315] 	| [-4.05133152] 	|
0.0 	| [3.0208821] 	| [-3.02088213] 	|
0.0 	| [4.041085] 	| [-4.04108477] 	|
0.0 	| [4.026568] 	| [-4.02656794] 	|
0.0 	| [3.6763072] 	| [-3.6763072] 	|
0.0 	| [4.0520115] 	| [-4.05201149] 	|
0.0 	| [4.0431495] 	| [-4.04314947] 	|
0.0 	| [3.8499174] 	| [-3.84991741] 	|
0.0 	| [3.691276] 	| [-3.69127607] 	|
0.0 	| [3.2115612] 	| [-3.2115612] 	|
0.0 	| [4.0077586] 	| [-4.00775862] 	|
0.0 	| [4.093283] 	| [-4.09328318] 	|
0.0 	| [3.8507848] 	| [-3.85078478] 	|
0.0 	| [4.0569324] 	| [-4.05693245] 	|
0.0 	| [4.0434637] 	| [-4.0

In [30]:
print("Actual\t|Predicted\t|Difference\t|")
for i in range(len(y_test)):  # Use len(y_test) instead of a fixed number
    print(y_test.iloc[i], "\t|", preds[i].round(), "\t|", y_test.iloc[i] - preds[i].round(), "\t|")


Streaming output truncated to the last 5000 lines.
5.0 	| [4.] 	| [1.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [3.] 	| [-3.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [3.] 	| [-3.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [3.] 	| [-3.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	| [-4.] 	|
0.0 	| [4.] 	|

In [31]:
print(mean_absolute_error(y_test,preds))
import numpy as np
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE:", rmse)


3.5087545379948617
RMSE: 3.6026029408112277


Declaring Function to compute states and action

In [32]:
import numpy as np

# Step 1: Clustering UAVs to Satellites (1-Based Indexing)
num_clusters = S
uav_indices = np.arange(1, U + 1)  # Start from 1 instead of 0
np.random.shuffle(uav_indices)  # Shuffle UAVs for random assignment
UAVs_per_cluster = np.array_split(uav_indices, num_clusters)

print("Clusters of UAVs:", [list(cluster) for cluster in UAVs_per_cluster])

# Step 2: Assign Ground Users to UAVs Randomly While Satisfying Capacity Constraints
GU_list = np.arange(1, N + 1)  # List of Ground Users (1-based indexing)
np.random.shuffle(GU_list)  # Shuffle GUs for random assignment

# Initialize User-UAV Association Matrix (U x N) (1-Based Indexing)
X_ui = np.zeros((U, N), dtype=int)

GU_assignment = {}  # Dictionary to store cluster-wise assignment
GU_counter = 0  # Start GU numbering from 1

# Assign users to UAVs randomly while ensuring max capacity constraint
for cluster_id, uav_list in enumerate(UAVs_per_cluster, start=1):  # Start cluster from 1
    GU_assignment[cluster_id] = {}
    for uav in uav_list:
        if GU_counter >= N:  # If all users are assigned, stop
            break

        # Determine how many users to assign (between 1 and M_u)
        num_GUs = min(np.random.randint(1, M_u + 1), N - GU_counter)

        # Assign GUs to UAV (1-Based Indexing)
        assigned_GUs = list(GU_list[GU_counter:GU_counter + num_GUs])
        GU_assignment[cluster_id][uav] = assigned_GUs

        # Update User-UAV association matrix (1-Based Indexing)
        for gu in assigned_GUs:
            X_ui[uav - 1, gu - 1] = 1  # Adjust for 0-based indexing in NumPy

        GU_counter += num_GUs

# Ensure all users are assigned (handling unassigned GUs)
all_assigned_GUs = sum(len(gus) for cluster in GU_assignment.values() for gus in cluster.values())
unassigned_GUs = N - all_assigned_GUs

if unassigned_GUs > 0:
    all_uavs = [uav for cluster in GU_assignment.values() for uav in cluster.keys()]
    np.random.shuffle(all_uavs)  # Shuffle UAVs for fair distribution
    for i in range(unassigned_GUs):
        selected_uav = all_uavs[i % len(all_uavs)]
        for cluster_id, uavs in GU_assignment.items():
            if selected_uav in uavs and len(uavs[selected_uav]) < M_u:
                GU_assignment[cluster_id][selected_uav].append(GU_list[GU_counter])
                X_ui[selected_uav - 1, GU_list[GU_counter] - 1] = 1  # Adjust for NumPy indexing
                GU_counter += 1
                break

# Print Results
print(f"Satellites (S): {S}, UAVs (U): {U}, Ground Users (N): {N}, Clusters: {num_clusters}, Max Users per UAV: {M_u}")

print("\nCluster Assignments:")
for cluster, uavs in GU_assignment.items():
    print(f"Cluster {cluster}:")
    for uav, gu_list in uavs.items():
        print(f"  UAV {uav}: {len(gu_list)} GUs -> {gu_list}")

print("\nUser-UAV Association Matrix (X_ui):")
print(X_ui)   # U x N


Clusters of UAVs: [[np.int64(7), np.int64(2), np.int64(8), np.int64(6)], [np.int64(1), np.int64(5), np.int64(3), np.int64(4)]]
Satellites (S): 2, UAVs (U): 8, Ground Users (N): 1000, Clusters: 2, Max Users per UAV: 20

Cluster Assignments:
Cluster 1:
  UAV 7: 20 GUs -> [np.int64(312), np.int64(191), np.int64(601), np.int64(996), np.int64(716), np.int64(830), np.int64(430), np.int64(105), np.int64(490), np.int64(586), np.int64(584), np.int64(792), np.int64(11), np.int64(753), np.int64(271), np.int64(44), np.int64(137), np.int64(321), np.int64(232), np.int64(64)]
  UAV 2: 20 GUs -> [np.int64(697), np.int64(53), np.int64(569), np.int64(620), np.int64(345), np.int64(779), np.int64(626), np.int64(537), np.int64(227), np.int64(87), np.int64(318), np.int64(801), np.int64(682), np.int64(260), np.int64(505), np.int64(492), np.int64(35), np.int64(703), np.int64(906), np.int64(575)]
  UAV 8: 20 GUs -> [np.int64(93), np.int64(406), np.int64(348), np.int64(733), np.int64(153), np.int64(82), np.int6

In [33]:
X_ui.shape   # ( U, N)

(8, 1000)

In [34]:

def content_popularity_prediction(N, F, u, X_ui, predicted_q_i_f):
    """
    Calculate total content popularity Q^f_u(t) for each content f for UAV 'u'.

    Parameters:
    - N (int): Total number of GUs
    - F (int): Number of content files
    - u (int): UAV ID for which popularity is calculated
    - X_ui (np.array): User-UAV Association Matrix (U x N)
    - predicted_q_i_f (np.array): Popularity matrix (N x F), each element q_i^f(t)

    Returns:
    - Q_f_u (np.array): Total popularity array for each content f for UAV 'u'
    """
    Q_f_u = np.zeros(F)  # Initialize total requests for UAV 'u'
    for f in range(F):
        temp = 0
        for i in range(N):  # Iterate over all users
            if X_ui[u-1][i] == 1:  # If user 'i' is associated with UAV 'u'
                temp += predicted_q_i_f[f+1][i+1]  # Add the user's requests
        Q_f_u[f] = temp

    return Q_f_u


# Compute content popularity for a specific UAV (e.g., UAV 1)
predicted_Q_f_u = content_popularity_prediction(N, F, 1, X_ui, predicted_q_i_f)

# Print Results
print(f"\nTotal Requests for UAV {1} (Q^f_u):")
print(predicted_Q_f_u)



Total Requests for UAV 1 (Q^f_u):
[80.197124   61.2290616  66.39785576 79.31187868 61.51745796 79.81522179
 66.55627584 71.33419514 45.54060733 71.18030357 80.03385544 40.41475677
 79.56076813 78.91268635 62.47362041 80.04588079 79.6732583  70.12928963
 62.34239388 45.79423344 78.22287035 81.6486311  70.17806149 80.31661034
 79.6888113  78.96756363 78.9717226  64.20640421 76.63462496 78.98382854
 78.98774576 80.32775211 70.93673134 80.33437204 79.70423937 79.00640678
 70.34733963 63.31613779 67.62470722 79.02026892 80.30700397 81.20510674
 79.03005362 59.68331289 79.55328941 79.71623802 80.5882926  79.69641829
 79.71862984 80.58961296 78.98130751 63.93123436 79.05898428 70.10680771
 79.06410551 61.836483   79.06900692 79.7235775  75.96530771 61.84107685
 80.08829546 79.08032179 64.35667229 68.23189735 64.42874336 76.60625792
 79.09031391 68.31199789 64.56812382 78.30711651 46.36344075 79.65879774
 79.3340354  79.72432089 79.66824865 76.57883739 70.9292016  79.10774851
 80.0901351  79.

In [35]:
predicted_Q_f_all_uav=[]
for u in range(0,U):
    predicted_Q_f_all_uav.append(content_popularity_prediction(N,F,u+1,X_ui,predicted_q_i_f))
print("array storing total requests count for all UAV set\n",predicted_Q_f_all_uav)



array storing total requests count for all UAV set
 [array([80.197124  , 61.2290616 , 66.39785576, 79.31187868, 61.51745796,
       79.81522179, 66.55627584, 71.33419514, 45.54060733, 71.18030357,
       80.03385544, 40.41475677, 79.56076813, 78.91268635, 62.47362041,
       80.04588079, 79.6732583 , 70.12928963, 62.34239388, 45.79423344,
       78.22287035, 81.6486311 , 70.17806149, 80.31661034, 79.6888113 ,
       78.96756363, 78.9717226 , 64.20640421, 76.63462496, 78.98382854,
       78.98774576, 80.32775211, 70.93673134, 80.33437204, 79.70423937,
       79.00640678, 70.34733963, 63.31613779, 67.62470722, 79.02026892,
       80.30700397, 81.20510674, 79.03005362, 59.68331289, 79.55328941,
       79.71623802, 80.5882926 , 79.69641829, 79.71862984, 80.58961296,
       78.98130751, 63.93123436, 79.05898428, 70.10680771, 79.06410551,
       61.836483  , 79.06900692, 79.7235775 , 75.96530771, 61.84107685,
       80.08829546, 79.08032179, 64.35667229, 68.23189735, 64.42874336,
       76.6

In [36]:
def rank_function(predicted_Q_f, predicted_Q_list):
    """
    Calculate the rank of a specific content f based on its estimated popularity Q_f.

    Args:
        predicted_Q_f (float): Popularity of a specific content f for a UAV.
        predicted_Q_list (list of float): Popularities of all contents for the same UAV.

    Returns:
        int: Rank of the content (lower rank means higher popularity).
    """
    return sum(predicted_Q_f <= predicted_Q_f_prime for predicted_Q_f_prime in predicted_Q_list)


# Dictionary to store rankings for all uav
rank_all_uav = {}

# Compute ranks for each UAV
for uav_idx , popularity_uav_list in enumerate(predicted_Q_f_all_uav):
    temp_dict = {}
    for f_idx, popularity_val in enumerate(popularity_uav_list):
        rank = rank_function(popularity_val,popularity_uav_list)
        temp_dict[f_idx] = rank
    rank_all_uav[uav_idx] = temp_dict


# Print Rankings
print("\nRanks of each content for all UAVs one by one:")
for uav, content_ranks_dict in rank_all_uav.items():
    print(f"\nUAV {uav}:")
    for content, rank in content_ranks_dict.items():
        print(f"  Content {content}, Pop'Value {predicted_Q_f_all_uav[uav][content]} : Rank {rank}")


Ranks of each content for all UAVs one by one:

UAV 0:
  Content 0, Pop'Value 80.19712400436401 : Rank 10
  Content 1, Pop'Value 61.22906160354614 : Rank 92
  Content 2, Pop'Value 66.39785575866699 : Rank 77
  Content 3, Pop'Value 79.31187868118286 : Rank 34
  Content 4, Pop'Value 61.51745796203613 : Rank 91
  Content 5, Pop'Value 79.81522178649902 : Rank 18
  Content 6, Pop'Value 66.55627584457397 : Rank 76
  Content 7, Pop'Value 71.33419513702393 : Rank 62
  Content 8, Pop'Value 45.54060733318329 : Rank 99
  Content 9, Pop'Value 71.1803035736084 : Rank 63
  Content 10, Pop'Value 80.03385543823242 : Rank 17
  Content 11, Pop'Value 40.414756774902344 : Rank 100
  Content 12, Pop'Value 79.5607681274414 : Rank 31
  Content 13, Pop'Value 78.91268634796143 : Rank 55
  Content 14, Pop'Value 62.47362041473389 : Rank 87
  Content 15, Pop'Value 80.04588079452515 : Rank 16
  Content 16, Pop'Value 79.67325830459595 : Rank 28
  Content 17, Pop'Value 70.1292896270752 : Rank 70
  Content 18, Pop'V

In [37]:
def tilde_cache_placement_from_rank(F, F_prime,rank_all_uav):
    """
    Generate ideal cache placement vector Γ̃_u(t)  using content rank information with predicted popularity and ignoring transmission constraint

    Parameters:
    - F (int): Total number of content files.
    - F_prime (int): Cache storage limit (maximum number of files that can be cached).
    - predicted_rank_dict (dict): Dictionary where keys are content indices and values are ranks (integers) based on predicted content popularity.

    Returns:
    - Gamma_u (np.array): Binary cache placement vector (0 or 1).
    """
    # Initialize cache vector with zeros
    Gamma = np.zeros((U,F), dtype=int)
    # Iterate through rank dictionary and cache content with rank <= F'
    for uav, per_uav_rank_list in rank_all_uav.items():
        for f, rank in per_uav_rank_list.items():
            if rank <= F_prime:
                Gamma[uav][f] = 1  # Cache this content

    return Gamma

# Compute cache placement vector
Gamma_tilde = tilde_cache_placement_from_rank(F, F_prime, rank_all_uav)

print("tilde Cache placement vector Γ(t):", Gamma_tilde)



tilde Cache placement vector Γ(t): [[1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0
  0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1
  1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0]
 [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0
  0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1
  1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0]
 [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0
  0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1
  1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0]
 [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0
  0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1
  1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0]
 [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0
  0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1

In [38]:
def generate_random_cache_placement(U, F, F_prime):
    """
    Generate a random cache placement matrix for time slot (t-1).

    Parameters:
    - U (int): Number of UAVs.
    - F (int): Total number of content files.
    - F_prime (int): Cache storage limit per UAV.

    Returns:
    - Gamma_prev (np.array): Random binary cache placement matrix (U x F).
    """
    Gamma_prev = np.zeros((U, F), dtype=int)

    for uav in range(U):
        cached_files = np.random.choice(F, F_prime, replace=False)  # Select F' unique files
        Gamma_prev[uav, cached_files] = 1  # Mark selected files as cached

    return Gamma_prev

Gamma_t_minus_1 = generate_random_cache_placement(U, F, F_prime)
print("Random cache placement at time slot (t-1):\n", Gamma_t_minus_1)


Random cache placement at time slot (t-1):
 [[1 1 1 1 0 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0
  0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 1
  1 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0]
 [0 1 1 1 0 0 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0
  1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0
  0 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 1 0]
 [1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 1 1
  0 0 0 1 1 0 0 1 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 1 0 0 1 0 1 0 0 1 0
  0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1]
 [0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 0 1 0 0
  1 1 0 1 1 1 0 1 1 0 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1 0]
 [1 0 0 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 1
  0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 

Define Cache load $ \Delta{\Gamma_u(t)} = | \tilde{\Gamma}_u(t)  - \Gamma_u(t-1) |$

In [39]:
def cache_update_load(Gamma_tilde, Gamma_prev):
    """
    Calculate the cache update load ΔΓ_u(t) for each UAV separately.

    Parameters:
    - Gamma_tilde (np.array): Ideal cache placement matrix Γ̃(t) (U x F).
    - Gamma_prev (np.array): Previous cache placement matrix Γ(t-1) (U x F).

    Returns:
    - Delta_Gamma (np.array): Array of size (U,) containing updates needed for each UAV.
    """

    # Compute element-wise absolute difference and sum per UAV (row-wise)
    Delta_Gamma = np.sum(np.abs(Gamma_tilde - Gamma_prev), axis=1)

    return Delta_Gamma  # Shape: (U,)

# Compute cache update load per UAV
Delta_Gamma = cache_update_load(Gamma_tilde, Gamma_t_minus_1)

# Print results
print("Cache update load ΔΓ_u(t) for each UAV:", Delta_Gamma)


Cache update load ΔΓ_u(t) for each UAV: [40 52 52 44 42 48 48 54]


In [40]:
def update_cache_vector( Gamma_t_minus_1, Delta_Gamma, L, F_prime):
    """
    Updates the cache vector for all UAVs based on popularity, previous cache, and transmission capacity.

    Args:
        Gamma_t_minus_1 (np.array): Previous cache matrix (U x F).
        Delta_Gamma (np.array): Cache update load vector (U,).
        L (np.array): Transmission capacity vector (U,).
        F_prime (int): Cache storage limit.

    Returns:
        np.array: Updated cache matrix (U x F).
    """
    U, F = Gamma_t_minus_1.shape  # Get number of UAVs and total files
    Gamma_t = np.zeros((U, F), dtype=int)  # Initialize updated cache matrix

    for u in range(U):  # Iterate over each UAV
        if L[u] >= Delta_Gamma[u]:
            Gamma_t[u] = Gamma_tilde[u]  # UAV can fully update cache
        else:
            # Step 1: Sort content by rank for the current UAV
            sorted_contents = sorted(rank_all_uav[u].keys(), key=lambda f: rank_all_uav[u][f])
            print(f"UAV {u+1}: Content indices with decreasing ranking: ", sorted_contents)

            # Step 2: Retain (F' - L[u]) files from previous cache based on popularity
            cached_indices = np.where(Gamma_t_minus_1[u] == 1)[0]
            print(f"UAV {u+1}: Previous cached state: ", cached_indices)

            retained_files = sorted(cached_indices, key=lambda f: rank_all_uav[u].get(f, float('inf')))[:F_prime - L[u]]
            print(f"UAV {u+1}: Files Retained: ", retained_files)

            # Step 3: Retrieve at most L[u] new files based on predicted ranks
            non_cached_indices = np.where(Gamma_t_minus_1[u] == 0)[0]
            new_files = sorted(non_cached_indices, key=lambda f: rank_all_uav[u].get(f, float('inf')))[:L[u]]
            print(f"UAV {u+1}: Files to bring from CSP: ", new_files)

            # Step 4: Update cache placement vector
            Gamma_t[u] = np.zeros(F, dtype=int)
            for f in retained_files + new_files:
                Gamma_t[u][f] = 1  # Mark content as cached

    return Gamma_t

# Example Usage:
Gamma_t = update_cache_vector(Gamma_t_minus_1, Delta_Gamma, L, F_prime)


UAV 1: Content indices with decreasing ranking:  [21, 80, 41, 49, 46, 33, 31, 23, 40, 0, 85, 78, 60, 90, 98, 15, 10, 5, 73, 57, 84, 48, 45, 34, 47, 83, 24, 16, 74, 71, 12, 44, 72, 3, 95, 92, 89, 82, 81, 79, 77, 66, 61, 56, 54, 52, 42, 39, 35, 30, 29, 50, 26, 25, 13, 69, 20, 28, 65, 75, 58, 7, 9, 97, 32, 76, 86, 36, 22, 17, 53, 91, 67, 63, 38, 6, 2, 99, 94, 87, 68, 64, 62, 27, 51, 37, 14, 18, 59, 55, 4, 1, 88, 93, 96, 43, 70, 19, 8, 11]
UAV 1: Previous cached state:  [ 0  1  2  3  5  6  8  9 10 15 20 21 24 26 27 29 30 34 38 41 44 50 54 56
 57 61 62 64 66 69 71 72 73 74 76 77 79 80 83 89 92 94 97]
UAV 1: Files Retained:  [np.int64(21), np.int64(80), np.int64(41), np.int64(0), np.int64(15), np.int64(10), np.int64(5), np.int64(73), np.int64(57), np.int64(34), np.int64(83), np.int64(24), np.int64(74), np.int64(71), np.int64(44)]
UAV 1: Files to bring from CSP:  [np.int64(49), np.int64(46), np.int64(33), np.int64(31), np.int64(23), np.int64(40), np.int64(85), np.int64(78), np.int64(60), np.i

In [41]:
# Updated Cache vector now which should be cached at UAV 'u' after all operations at time slice 't'
for uav in range(0,U):
    print("UAV",uav+1," : ",Gamma_t[uav])

UAV 1  :  [1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 1 1 1 0
 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1
 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0]
UAV 2  :  [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0
 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1
 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0]
UAV 3  :  [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0
 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 1
 1 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0]
UAV 4  :  [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0
 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 0 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0]
UAV 5  :  [1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 0 0
 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 

Since we just calculated Cache placement vector for all UAVs in a 2-D matrix (U x F), Now we calculate Cache hit ratio of all uavs and store them in a vector of size U

$$
\mathcal{H}_{u}(t) = \frac{\sum\limits_{f \in \mathcal{F}} \Big( \Gamma_{u,f}(t) + (1 - \Gamma_{u,f}(t)) \max\limits_{u' \in \mathcal{U_s}} \Gamma_{u',f}(t) \Big) \hat{Q}^f_{u}(t)}{\sum\limits_{f' \in \mathcal{F}} \hat{Q}^{f'}_{u}(t)}
$$



In [42]:
def compute_cache_hit_ratio(Gamma_t, Q_f_u, UAVS_per_cluster):
    """
    Computes the cache hit ratio H_u(t) for each UAV.

    Args:
        Gamma_t (np.array): Binary cache placement matrix (U x F).
        Q_f_u (np.array): Request probability matrix (U x F).
        UAVS_per_cluster (dict): Dictionary mapping cluster_id -> list of UAVs in that cluster.

    Returns:
        np.array: Cache hit ratio vector for all UAVs.
    """

    H = np.zeros(U)  # Cache hit ratio for each UAV
    cluster_dict = {idx: cluster for idx, cluster in enumerate(UAVS_per_cluster)}

    for u in range(1,U+1):  # Iterate over UAVs
        cluster_id = next((cid for cid, members in cluster_dict.items() if u in members), None)

        if cluster_id is None:  # Handle missing UAV
            print(f"Warning: UAV {u} not found in any cluster!")
            continue  # Skip to next UAV
        # Find all UAVs in the same cluster
        U_s = UAVS_per_cluster[cluster_id]
        # print(U_s)

        # Compute the numerator
        numerator = sum(
            (Gamma_t[u-1][f] + (1 - Gamma_t[u-1][f]) * (max(Gamma_t[u_prime-1][f] for u_prime in U_s)))
            * Q_f_u[u-1][f]
            for f in range(F)
        )

        # Compute the denominator
        denominator = sum(Q_f_u[u-1][f] for f in range(F))

        # Avoid division by zero
        H[u-1] = numerator / denominator if denominator != 0 else 0

    return H

H = compute_cache_hit_ratio(Gamma_t, predicted_Q_f_all_uav, UAVs_per_cluster)

# Print results
print("Cache Hit Ratio H(t) => ")
for u in range(U):
    print(f"UAV {u+1}: {H[u]:.4f}")


Cache Hit Ratio H(t) => 
UAV 1: 0.6081
UAV 2: 0.5328
UAV 3: 0.6061
UAV 4: 0.6118
UAV 5: 0.6087
UAV 6: 0.5343
UAV 7: 0.5382
UAV 8: 0.5383


We had now done for Predicted popularity values but now since we need to define that how better had our model performed well as compared to actual popularity values, Therefore, We now do as following :
1. Generate a random actual popularity matrix similar to predicted popularity matrix
2. Find similar ideal matrices based on acual popularity values, as done using predicted popularity values
3. Find Maximal Cache hit ratio
4. and finally define system regret


In [43]:
actual_q_i_f        # shape = (N,F)

actual_Q_f_u = content_popularity_prediction(N,F,0,X_ui,actual_q_i_f)  # just an example to show for one UAV

# Print Results
print(f"\nTotal Requests for UAV {0} (Q^f_u):")
print(actual_Q_f_u)


Total Requests for UAV 0 (Q^f_u):
[19.  6.  8.  0.  0. 12.  0.  0.  0. 15.  7.  0.  0.  0.  0. 11.  8.  4.
  0.  1. 22.  7.  3. 11.  8.  0.  0.  4. 10.  0.  6. 22.  0.  8.  0. 25.
  0.  0.  8.  0.  3.  4.  0.  2.  8.  0. 17.  6.  0. 35.  0.  0.  0.  0.
  3.  0.  0.  0.  0.  7.  0.  9.  0.  0.  0.  0.  0.  0.  0.  8.  0.  0.
  5.  0.  0.  0.  0.  0.  3.  0.  5.  0.  0.  0.  2.  0.  0.  0.  7.  0.
  0.  0.  4. 12.  0.  0.  0.  0.  4.  0.]


In [44]:
actual_Q_f_all_uav=[]
for u in range(0,U):
    actual_Q_f_all_uav.append(content_popularity_prediction(N,F,u+1,X_ui,actual_q_i_f))

print("array storing total requests count for all UAV set",actual_Q_f_all_uav)


array storing total requests count for all UAV set [array([37., 12., 13.,  7., 14., 16., 15.,  0.,  2., 12., 29.,  0.,  0.,
        5.,  3., 10., 11.,  6.,  5.,  4., 26., 12.,  5., 10., 13.,  7.,
        3.,  4.,  7.,  3.,  6., 30.,  0., 31.,  4., 26.,  0.,  0., 26.,
        0., 13.,  1.,  0.,  7., 19.,  8., 19.,  4.,  0., 27.,  0.,  0.,
        0.,  0.,  0.,  0.,  3., 12.,  0.,  9.,  1., 17.,  1.,  4.,  3.,
        0.,  0.,  0., 14., 12.,  1.,  0.,  4.,  0.,  1.,  4.,  0.,  4.,
        7.,  4.,  4.,  2.,  0.,  0.,  0.,  7.,  0., 12.,  7.,  0.,  7.,
        3.,  7., 14.,  0.,  0.,  0.,  4.,  2.,  5.]), array([22.,  5.,  3.,  0.,  0.,  4.,  7.,  0.,  0.,  4.,  0.,  3.,  0.,
        0.,  5.,  5., 17.,  4.,  2.,  0., 20.,  4.,  0.,  4.,  5.,  0.,
        0.,  4.,  5.,  0.,  0.,  4.,  0., 16.,  5.,  3.,  0.,  0., 15.,
        0.,  0.,  0.,  0.,  2.,  5.,  0., 12.,  1.,  0., 19.,  0.,  2.,
        0.,  3.,  7.,  0.,  0.,  6.,  0.,  3.,  0.,  3.,  0.,  0.,  0.,
        0.,  0.,  0.,  3., 14.

In [45]:
def actual_rank_function(actual_Q_f, actual_Q_list):
    """
    Calculate the rank of a specific content f based on its estimated popularity Q_f.

    Args:
        actual_Q_f (float): Popularity of a specific content f for a UAV.
        actual_Q_list (list of float): Popularities of all contents for the same UAV.

    Returns:
        int: Rank of the content (lower rank means higher popularity).
    """
    return sum(actual_Q_f <= actual_Q_f_prime for actual_Q_f_prime in actual_Q_list)


# Dictionary to store rankings for all uav
actual_rank_all_uav = {}

# Compute ranks for each UAV
for uav_idx , popularity_uav_list in enumerate(actual_Q_f_all_uav):
    temp_dict = {}
    for f_idx, popularity_val in enumerate(popularity_uav_list):
        rank = actual_rank_function(popularity_val,popularity_uav_list)
        temp_dict[f_idx] = rank
    actual_rank_all_uav[uav_idx] = temp_dict


# Print Rankings
print("\nRanks of each content for all UAVs one by one:")
for uav, content_ranks_dict in actual_rank_all_uav.items():
    print(f"\nUAV {uav+1}:")
    for content, rank in content_ranks_dict.items():
        print(f"  Content {content}, Pop'Value {actual_Q_f_all_uav[uav][content]} : Rank {rank}")


Ranks of each content for all UAVs one by one:

UAV 1:
  Content 0, Pop'Value 37.0 : Rank 1
  Content 1, Pop'Value 12.0 : Rank 25
  Content 2, Pop'Value 13.0 : Rank 19
  Content 3, Pop'Value 7.0 : Rank 39
  Content 4, Pop'Value 14.0 : Rank 16
  Content 5, Pop'Value 16.0 : Rank 12
  Content 6, Pop'Value 15.0 : Rank 13
  Content 7, Pop'Value 0.0 : Rank 100
  Content 8, Pop'Value 2.0 : Rank 65
  Content 9, Pop'Value 12.0 : Rank 25
  Content 10, Pop'Value 29.0 : Rank 4
  Content 11, Pop'Value 0.0 : Rank 100
  Content 12, Pop'Value 0.0 : Rank 100
  Content 13, Pop'Value 5.0 : Rank 45
  Content 14, Pop'Value 3.0 : Rank 62
  Content 15, Pop'Value 10.0 : Rank 28
  Content 16, Pop'Value 11.0 : Rank 26
  Content 17, Pop'Value 6.0 : Rank 41
  Content 18, Pop'Value 5.0 : Rank 45
  Content 19, Pop'Value 4.0 : Rank 56
  Content 20, Pop'Value 26.0 : Rank 8
  Content 21, Pop'Value 12.0 : Rank 25
  Content 22, Pop'Value 5.0 : Rank 45
  Content 23, Pop'Value 10.0 : Rank 28
  Content 24, Pop'Value 13.0 

In [46]:
# creating Xi (similar to Gamma), which is a cache placement vector according to actual popularity value of content )

''' we have already created function for that for calculating gamma
so now directly calling it to have \Xi(t) but
will pass actual popularity content matrix as an arguement '''

xi_tilde = tilde_cache_placement_from_rank(F,F_prime, actual_rank_all_uav)

print("tilde Cache placement vector Xi(t) based on actual popularity : ", xi_tilde)

tilde Cache placement vector Xi(t) based on actual popularity :  [[1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1
  0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0
  0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0]
 [1 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1 0 1 1 1
  0 0 1 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 1 0 1
  0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0]
 [1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0
  0 0 1 0 0 0 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0
  0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 1
  0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 1 0 0
  1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1]
 [1 1 1 0 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1
  0 0 1 0 0 0 1 1 1 0 1 1 0 1 0 0

Define Cache load $ \Delta{\xi_u(t)} = | \tilde{\xi}_u(t)  - \Gamma_u(t-1) |$

In [47]:
Delta_xi = cache_update_load(xi_tilde,Gamma_t_minus_1)
# Print results
print("Cache update load Δxi_u(t) for each UAV:", Delta_xi)


Cache update load Δxi_u(t) for each UAV: [46 48 46 46 43 45 57 52]


In [48]:
xi_t = update_cache_vector( Gamma_t_minus_1, Delta_xi, L, F_prime)


UAV 1: Content indices with decreasing ranking:  [21, 80, 41, 49, 46, 33, 31, 23, 40, 0, 85, 78, 60, 90, 98, 15, 10, 5, 73, 57, 84, 48, 45, 34, 47, 83, 24, 16, 74, 71, 12, 44, 72, 3, 95, 92, 89, 82, 81, 79, 77, 66, 61, 56, 54, 52, 42, 39, 35, 30, 29, 50, 26, 25, 13, 69, 20, 28, 65, 75, 58, 7, 9, 97, 32, 76, 86, 36, 22, 17, 53, 91, 67, 63, 38, 6, 2, 99, 94, 87, 68, 64, 62, 27, 51, 37, 14, 18, 59, 55, 4, 1, 88, 93, 96, 43, 70, 19, 8, 11]
UAV 1: Previous cached state:  [ 0  1  2  3  5  6  8  9 10 15 20 21 24 26 27 29 30 34 38 41 44 50 54 56
 57 61 62 64 66 69 71 72 73 74 76 77 79 80 83 89 92 94 97]
UAV 1: Files Retained:  [np.int64(21), np.int64(80), np.int64(41), np.int64(0), np.int64(15), np.int64(10), np.int64(5), np.int64(73), np.int64(57), np.int64(34), np.int64(83), np.int64(24), np.int64(74), np.int64(71), np.int64(44)]
UAV 1: Files to bring from CSP:  [np.int64(49), np.int64(46), np.int64(33), np.int64(31), np.int64(23), np.int64(40), np.int64(85), np.int64(78), np.int64(60), np.i

In [49]:
# Updated ideal Cache vector now which should be cached at UAV 'u' after all operations at time slice 't'
print("Maximal-Hit-Ratio Content Placement Vector ξ_u(t):")
for uav in range(0,U):
    print("UAV",uav+1," : ",xi_t[uav])


Maximal-Hit-Ratio Content Placement Vector ξ_u(t):
UAV 1  :  [1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 1 1 1 0
 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1
 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0]
UAV 2  :  [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0
 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1
 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0]
UAV 3  :  [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0
 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 1
 1 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0]
UAV 4  :  [1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0
 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 0 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0]
UAV 5  :  [1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 0 0
 0 0 1 1

Since we just calculated Cache placement vector for all UAVs based on actual popularity in a 2-D matrix (U x F), Now we calculate Cache hit ratio of all uavs and store them in a vector of size U

$$
\mathcal{H}^{\text{max}}_{u}(t) = \frac{\sum\limits_{f \in \mathcal{F}} \Big( \xi_{u,f}(t) + (1 - \xi_{u,f}(t)) \max\limits_{u' \in \mathcal{U_s}} \xi_{u',f}(t) \Big) Q^f_{u}(t)}{\sum\limits_{f' \in \mathcal{F}} Q^{f'}_{u}(t)}
$$



In [50]:
H_max = compute_cache_hit_ratio(xi_t, actual_Q_f_all_uav, UAVs_per_cluster)
# Print results
print("Max_Cache Hit Ratio H^(max)_u(t) => ")
for u in range(U):
    print(f"UAV {u+1}: {H_max[u]:.4f}")





Max_Cache Hit Ratio H^(max)_u(t) => 
UAV 1: 0.6361
UAV 2: 0.5893
UAV 3: 0.7963
UAV 4: 0.6250
UAV 5: 0.5478
UAV 6: 0.7258
UAV 7: 0.5754
UAV 8: 0.6667


In [51]:
def system_regret_func(H_max, H):
    temp = []
    for i in range(U):
        temp.append(1)
    regret_vector = []
    for uav in range(0,U):
        regret_vector.append(temp[uav] - H[uav])
    return regret_vector



# Values may come negative but it will always be positive only!
# we have randomly generated without algorithm for that reason negavtive are coming

# so, taking mod to fix the issue because anyway positive values won't be affected

system_regret = system_regret_func(H_max, H)
print("System regret = ", sum(system_regret))



System regret =  3.42184067061685


In [52]:
# Constants
# d_ui = 150  # Distance between primary UAV u and GU i (in meters)
d_ks = 780000   # Distance between CSP k and satellite s (in meters)
d_su = 780000   # Distance between satellite s and UAV u (in meters)
# d_uprime_u = 200  # Distance between secondary UAV u' and primary UAV u (in meters)

c = 3e8  # Speed of light in m/s

# Nuprime_u = 5  # Number of files transferred from UAV u' to UAV u
Nk_s = 5    # Number of files transferred between CSP k and satellite s
Ns_u = 3    # Number of files transferred between satellite s and UAV u


# 10 ** ((transmit_power_sat_max - 30) / 10)  formula dbm to watts
# p_ui = 10 ** ((37 - 30) / 10)  # Transmission power of UAV u (in Watts)
# p_uprime_u = 10 ** ((37 - 30) / 10)   # Transmission power of UAV u' (in Watts)
ps_u = 10 ** ((50 - 30) / 10)
pk_s = 10 ** ((50 - 30) / 10)

p_max = 50  # Max power for UAVs

alpha_1 = 2  # path loss exponent between CSP and Satellite
alpha_2 = 2  # path loss exponent between satellite and UAV
# alpha_3 = 2  # path loss exponent between primary uav and secondary uav
# alpha_4 = 2 # path loss exponent between primary uav and ground user

g_k_s = 0.64  # fading factor between CSP and Satellite
g_s_u =  0.68783 # fading factor between satellite and UAV
# g_uprime_u = 0.86  # fading factor between primary uav and secondary uav
# g_u_i = 0.69  # fading factor between primary uav and ground user

B1 = 10*1000000   #transmission bandwidth between CSP and Satellite
B2 = 10*1000000  #transmission bandwidth between satellite and UAV
B3 = 10*1000000  #transmission bandwidth between primary uav and secondary uav
# B4 = 10*1000000  #transmission bandwidth between primary uav and ground user

def dbm_to_watts(dbm):
    return 10 ** ((dbm - 30) / 10)

# sigma_k = dbm_to_watts(-174 + 10 * math.log10(B1))
sigma_s = dbm_to_watts(-174 + 10 * math.log10(B1))
sigma_u = dbm_to_watts(-174 + 10 * math.log10(B2))
# sigma_i = dbm_to_watts(-174 + 10 * math.log10(B4))


Z = 100 * 1024    # File size (bits or packets)
Y = 10    # Fixed caching cost
P_k = 0.05  # Price per bit

cost_constant = 10**(-8)


In [53]:
def euclidean_distance(pos1, pos2):
    return np.linalg.norm(np.array(pos1) - np.array(pos2))

def channel_gain(d, alpha, g):
    h= np.sqrt(g* (d ** -alpha))
    return h

def calculate_data_rate( B, p, CG, sigma):
    numerator = p * CG
    rate = B * np.log2(1 + numerator / (sigma**2))

    return rate

# def compute_latency_primary(d_ui, R_ui):
#     return (2 * d_ui / c) + (Z / R_ui)

# def compute_latency_secondary(d_uprime_u, d_ui, Nuprime_u, R_uprime_u, R_ui):
#     return (2 * (d_uprime_u + d_ui) / c) + (Nuprime_u * Z / R_uprime_u) + (Z / R_ui)

# def compute_latency_transfer(d_uprime_u, Nuprime_u, R_uprime_u):
#     return (2 * d_uprime_u / c) + (Nuprime_u * Z / R_uprime_u)

def compute_latency_Tks(d_ks, Nk_s, Rk_s):
    return (2 * d_ks / c) + (Nk_s * Z / Rk_s)

def compute_latency_Tsu(d_su, Ns_u, Rs_u):
    return (2 * d_su / c) + (Ns_u * Z / Rs_u)

# def compute_latency_Tki(d_ks, d_su, d_ui, Ns_u, Nk_s, Rs_u, Rk_s, R_ui):
#     return (2 * (d_ks + d_su + d_ui) / c) + (Ns_u * Z / Rs_u) + (Nk_s * Z / Rk_s) + (Z / R_ui)



In [54]:
# # For Primary UAV
# def compute_D_ui(R_ui, T_ui):
#     return R_ui * T_ui

# # For secondary UAV
# def compute_D_uprime_i(R_ui, T_ui, R_uprime_u, T_uprime_u):
#     return R_ui * T_ui + R_uprime_u * T_uprime_u

# # For CSP route
# def compute_D_k_i(R_ui, T_ui, R_su, T_su, R_ks, T_ks):
#     return R_ui * T_ui + R_su * T_su + R_ks * T_ks

# For CSP to UAV only
def compute_D_k_u( R_su, T_su, R_ks, T_ks):
    return  R_su * T_su + R_ks * T_ks


In [55]:

# # 1. Cost via Primary UAV
# def cost_primary_uav(P_k, R_ui, T_ui):
#     D_ui = compute_D_ui(R_ui, T_ui)
#     return P_k * D_ui + Y

# # 2. Cost via Secondary UAV
# def cost_secondary_uav(P_k, R_ui, T_ui, R_uprime_u, T_uprime_u):
#     D_uprime_i = compute_D_uprime_i(R_ui, T_ui, R_uprime_u, T_uprime_u)
#     return P_k * D_uprime_i + Y

# 3. Cost via CSP path
def cost_csp_path(P_k, R_su, T_su, R_ks, T_ks):
    D_k_u= compute_D_k_u( R_su, T_su, R_ks, T_ks)
    return P_k * D_k_u


def compute_csp_cost_for_uav(u, F, Gamma_t, Gamma_2_t, UAVs_per_cluster, C_k_u):
    cluster_uav_index = u + 1
    cluster_uavs = []
    for cluster in UAVs_per_cluster:
        if cluster_uav_index in cluster:
            cluster_uavs = cluster
            break

    cost = 0.0
    for f in range(F):
        gamma_2 = Gamma_2_t[u][f]           # UAV u fetches file f from CSP
        gamma_local = Gamma_t[u][f]         # UAV u already has file f
        gamma_cluster_max = max(Gamma_t[u_prime - 1][f] for u_prime in cluster_uavs)

        if gamma_2 == 1:
            cost += C_k_u

    return cost


In [56]:

def compute_total_cost( Gamma_t, Gamma_2_u, UAVs_per_cluster, F, U):
    total_cost = 0
    individual_cost_vector_uav = []
    for u in range(U):
        R_su = calculate_data_rate(B2,ps_u, channel_gain(d_su,alpha_2, g_s_u), sigma_u )
        # print(R_su)
        R_ks = calculate_data_rate(B1,pk_s, channel_gain(d_ks,alpha_1, g_k_s), sigma_s )
        # print(R_ks)

        T_su = compute_latency_Tsu(d_su, Nk_s, R_su)
        # print(T_su)
        T_ks = compute_latency_Tks(d_ks, Nk_s, R_ks)
        # print(T_ks)

        C_k_u = cost_csp_path(P_k, R_su, T_su, R_ks, T_ks)
        # print(C_k_u)
        cost_i_k = compute_csp_cost_for_uav(u, F, Gamma_t, Gamma_2_u , UAVs_per_cluster, C_k_u)
        # print(cost_i_k)
        individual_cost_vector_uav.append(cost_i_k)
        total_cost += cost_i_k
    # print("TOTOAL_OCST= = ",total_cost)
    # print("Individual cost for each uav = ", individual_cost_vector_uav)
    return total_cost, individual_cost_vector_uav


In [57]:
compute_total_cost(X_ui,Gamma_t , UAVs_per_cluster, F, U)

(np.float64(153208278.859669),
 [np.float64(19151034.857458625),
  np.float64(19151034.857458625),
  np.float64(19151034.857458625),
  np.float64(19151034.857458625),
  np.float64(19151034.857458625),
  np.float64(19151034.857458625),
  np.float64(19151034.857458625),
  np.float64(19151034.857458625)])

In [58]:
def joint_problem(system_regret, total_cost):
    min_cost = min(total_cost)
    max_cost = max(total_cost)

    joint_Reward_total = 0
    for i in range(len(total_cost)):
        if max_cost == min_cost:
            normalized_cost = 0  # or use some default value or skip normalization
        else:
            normalized_cost = (total_cost[i] - min_cost) / (max_cost - min_cost)

        joint_Reward_total += (system_regret[i] + normalized_cost)

    return joint_Reward_total


In [59]:
def generate_rank_all_uav(predicted_Q_f_all_uav):
    def rank_function(predicted_Q_f, predicted_Q_list):
        return sum(predicted_Q_f <= predicted_Q_f_prime for predicted_Q_f_prime in predicted_Q_list)

    rank_all_uav = {}
    for uav_idx, popularity_uav_list in enumerate(predicted_Q_f_all_uav):
        temp_dict = {}
        for f_idx, popularity_val in enumerate(popularity_uav_list):
            rank = rank_function(popularity_val, popularity_uav_list)
            temp_dict[f_idx] = rank
        rank_all_uav[uav_idx] = temp_dict

    return rank_all_uav


In [60]:
def to_bring_files(curr_cache, Gamma_t_minus_1, predicted_Q_f, L):
    U, F = curr_cache.shape
    rank_all_uav = generate_rank_all_uav(predicted_Q_f)
    new_files_matrix = np.zeros((U, F), dtype=int)

    for u in range(U):
        # Files newly added: curr_cache - prev_cache = 1
        new_file_indices = [f for f in range(F) if (curr_cache[u][f] - Gamma_t_minus_1[u][f]) == 1]

        # Sort the new files based on rank (lower rank = higher popularity)
        sorted_new_files = sorted(new_file_indices, key=lambda f: rank_all_uav[u].get(f, float('inf')))

        # Pick top L[u] files to bring
        top_files = sorted_new_files[:L[u]]

        # Mark in output matrix
        for f in top_files:
            new_files_matrix[u][f] = 1

    return new_files_matrix


**Environment building for DRL**

In [61]:
import numpy as np
import random
import tensorflow as tf
import gym
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Add, Lambda
from tensorflow.keras.optimizers import Adam
from collections import deque

#  Enable GPU acceleration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Using GPU: {gpus[0]}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected, running on CPU.")




No GPU detected, running on CPU.


In [62]:
import gym
from gym import spaces
import numpy as np

class UAVCacheEnv(gym.Env):
    def __init__(self, num_users=N, cache_size=F_prime, num_contents=F, num_uavs= U):  # Provide default values
        super(UAVCacheEnv, self).__init__()

        self.num_users = num_users
        self.cache_size = cache_size
        self.num_contents = num_contents
        self.num_uavs = num_uavs
        # Transmission constraint (L) is a single integer
        self.transmission_constraints = L

        self.state_size = 4*(U*F) + U
        self.action_size = U*F  # Number of binary cache choices

        # Define observation space
        self.observation_space = gym.spaces.Dict({
            "predicted_Q_f": gym.spaces.Box(low=0, high=np.inf,
                                    shape=(self.num_uavs, self.num_contents), dtype=np.float32),
            "Gamma_t": gym.spaces.MultiBinary((self.num_uavs, self.num_contents)),   # Current Cache Status based on predicted popularities
            "Xi_t": gym.spaces.MultiBinary((self.num_uavs, self.num_contents)),   #Ideal Current cache Status based on actual popularities
            "Gamma_t_minus_1": gym.spaces.MultiBinary((self.num_uavs, self.num_contents)),    # Previous Cache Status based on predicted popularities[WILL BE REAL]
            "transmission_constraints": gym.spaces.Discrete(self.num_uavs)    # Transmission contraint vector for uavs
        })

        # Define action space
        self.action_space = gym.spaces.MultiBinary((self.num_uavs, self.num_contents))

        self.reset()  # Ensure state variables are initialized

    def reset(self):
        """ Resets the environment to its initial state. """
        self.predicted_Q_f = predicted_Q_f_all_uav
        self.Gamma_t = np.zeros((self.num_uavs, self.num_contents), dtype=np.int32)
        self.Xi_t = xi_t
        self.Gamma_t_minus_1 = np.zeros((self.num_uavs, self.num_contents), dtype=np.int32)
        self.transmission_constraints = L
        return self._get_state()

    def _get_state(self):
        """ Returns the current state of the environment. """
        return {
            "predicted_Q_f":self.predicted_Q_f,
            "Gamma_t_minus_1": self.Gamma_t_minus_1,
            "Gamma_t": self.Gamma_t,
            "Xi_t": self.Xi_t,
            "transmission_constraints": self.transmission_constraints
        }


    def calculate_reward(self, action):
        H = compute_cache_hit_ratio(action, self.predicted_Q_f , UAVs_per_cluster)
        H_max = compute_cache_hit_ratio(self.Xi_t, actual_Q_f_all_uav, UAVs_per_cluster)
        system_regret = system_regret_func(H_max , H)
        # print("Predicted cache hit =>" ,H)
        # print("MAX cache hit =>" ,H_max)
        # return -sum(system_regret)
        # print("System regret" ,system_regret)
        brought_content = to_bring_files(action, self.Gamma_t_minus_1, self.predicted_Q_f , self.transmission_constraints)
        # print("content ot be broght = ", brought_content)
        total_cost, individual_cost_each_uav = compute_total_cost( self.Gamma_t, brought_content , UAVs_per_cluster, F, U)
        # print("total_cost_indivually = ", individual_cost_each_uav)

        #min max normalisation


        reward = -joint_problem(system_regret, individual_cost_each_uav)
        # print(reward)
        return reward, H ,H_max,  system_regret , total_cost

    def step(self, action):
        """ Takes an action and updates the environment. """
        done = False
        # print("IAM ACTION *****\t",len(action), action)
        self.Gamma_t_minus_1 = np.array(self.Gamma_t)  # update previous cache to current cache
        self.Gamma_t = np.array(action)  # Update Current cache status to actino taken

        # Reward: Minimize cache mismatch loss
        reward,H ,H_max,  system_regret , total_cost  = self.calculate_reward(np.array(action))

        return self._get_state(), reward, H ,H_max,  system_regret , total_cost , done, {}

    def render(self):
        """ Prints the current environment state. """
        # print(f"Gamma(t-1): {self.Gamma_t_minus_1}")
        # print(f"Gamma(t): {self.Gamma_t}")
        # print(f"Xi(t): {self.Xi_t}")
        # print(f"Transmission Constraint (L): {self.transmission_constraints}")

# Example Usage
env = UAVCacheEnv()
state = env.reset()
print("\nInitial State:")
env.render()



Initial State:


In [63]:
def select_action(state_dict):
    predicted_Q_f = state_dict["predicted_Q_f"]
    Gamma_t_minus_1 = state_dict["Gamma_t_minus_1"]
    Gamma_t = state_dict["Gamma_t"]
    Xi_t = state_dict["Xi_t"]
    transmission_constraints = state_dict["transmission_constraints"]

    # Dictionary to store rankings for all uav
    rank_all_uav = {}

    # Compute ranks for each UAV
    for uav_idx , popularity_uav_list in enumerate(predicted_Q_f):
        temp_dict = {}
        for f_idx, popularity_val in enumerate(popularity_uav_list):
            rank = rank_function(popularity_val,popularity_uav_list)
            temp_dict[f_idx] = rank
        rank_all_uav[uav_idx] = temp_dict

    Gamma_tilde = tilde_cache_placement_from_rank(F, F_prime, rank_all_uav)

    Gamma_t_minus_1 = Gamma_t_minus_1
    Delta_Gamma = cache_update_load(Gamma_tilde, Gamma_t_minus_1)
    Gamma_t = update_cache_vector(Gamma_t_minus_1, Delta_Gamma, L, F_prime)


    return Gamma_t


In [64]:
select_action(state)

UAV 1: Content indices with decreasing ranking:  [21, 80, 41, 49, 46, 33, 31, 23, 40, 0, 85, 78, 60, 90, 98, 15, 10, 5, 73, 57, 84, 48, 45, 34, 47, 83, 24, 16, 74, 71, 12, 44, 72, 3, 95, 92, 89, 82, 81, 79, 77, 66, 61, 56, 54, 52, 42, 39, 35, 30, 29, 50, 26, 25, 13, 69, 20, 28, 65, 75, 58, 7, 9, 97, 32, 76, 86, 36, 22, 17, 53, 91, 67, 63, 38, 6, 2, 99, 94, 87, 68, 64, 62, 27, 51, 37, 14, 18, 59, 55, 4, 1, 88, 93, 96, 43, 70, 19, 8, 11]
UAV 1: Previous cached state:  []
UAV 1: Files Retained:  []
UAV 1: Files to bring from CSP:  [np.int64(21), np.int64(80), np.int64(41), np.int64(49), np.int64(46), np.int64(33), np.int64(31), np.int64(23), np.int64(40), np.int64(0), np.int64(85), np.int64(78), np.int64(60), np.int64(90), np.int64(98), np.int64(15), np.int64(10), np.int64(5), np.int64(73), np.int64(57), np.int64(84), np.int64(48), np.int64(45), np.int64(34), np.int64(47), np.int64(83), np.int64(24), np.int64(16)]
UAV 2: Content indices with decreasing ranking:  [21, 80, 41, 49, 46, 31, 3

array([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 0, 0,

In [65]:
# Running the test
env = UAVCacheEnv()
state = env.reset()
print("\nInitial State:")
env.render()


action = select_action(state)  # Random binary cache decision
print("ACTION: ", action)
# Taking a step
next_state, reward, H ,H_max,  system_regret , total_cost ,done, _ = env.step(action)


print("\nAfter Step:")
env.render()
print(f"Reward: {reward}")

# -----
print("second ieration=========")
env.render()


action = select_action(state)  # Random binary cache decision
print("ACTION: ", action)
# Taking a step
next_state, reward, H ,H_max,  system_regret , total_cost ,done, _ = env.step(action)


print("\nAfter Step:")
env.render()
print(f"Reward: {reward}")

print("second ieration=========")
env.render()


action = select_action(state) # Random binary cache decision
print("ACTION: ", action)
# Taking a step
next_state, reward,  H ,H_max,  system_regret , total_cost , done, _ = env.step(action)


print("\nAfter Step:")
env.render()
print(f"Reward: {reward}")


Initial State:
UAV 1: Content indices with decreasing ranking:  [21, 80, 41, 49, 46, 33, 31, 23, 40, 0, 85, 78, 60, 90, 98, 15, 10, 5, 73, 57, 84, 48, 45, 34, 47, 83, 24, 16, 74, 71, 12, 44, 72, 3, 95, 92, 89, 82, 81, 79, 77, 66, 61, 56, 54, 52, 42, 39, 35, 30, 29, 50, 26, 25, 13, 69, 20, 28, 65, 75, 58, 7, 9, 97, 32, 76, 86, 36, 22, 17, 53, 91, 67, 63, 38, 6, 2, 99, 94, 87, 68, 64, 62, 27, 51, 37, 14, 18, 59, 55, 4, 1, 88, 93, 96, 43, 70, 19, 8, 11]
UAV 1: Previous cached state:  []
UAV 1: Files Retained:  []
UAV 1: Files to bring from CSP:  [np.int64(21), np.int64(80), np.int64(41), np.int64(49), np.int64(46), np.int64(33), np.int64(31), np.int64(23), np.int64(40), np.int64(0), np.int64(85), np.int64(78), np.int64(60), np.int64(90), np.int64(98), np.int64(15), np.int64(10), np.int64(5), np.int64(73), np.int64(57), np.int64(84), np.int64(48), np.int64(45), np.int64(34), np.int64(47), np.int64(83), np.int64(24), np.int64(16)]
UAV 2: Content indices with decreasing ranking:  [21, 80, 4

In [66]:
def flatten_state(state_dict):
    """
    Converts the state dictionary into a flat NumPy array.
    """
    predicted_Q_f = np.array(state_dict["predicted_Q_f"]).flatten()  # Shape (U, F) → (U * F,)
    Gamma_t_minus_1 = np.array(state_dict["Gamma_t_minus_1"]).flatten()  # Shape (U, F) → (U * F,)
    Gamma_t = np.array(state_dict["Gamma_t"]).flatten()  # Shape (U, F) → (U * F,)
    Xi_t = np.array(state_dict["Xi_t"]).flatten()  # Shape (U, F) → (U * F,)
    transmission_constraints = np.array(state_dict["transmission_constraints"]).flatten()  # Shape (U,) → (U,)

    return np.concatenate([predicted_Q_f, Gamma_t_minus_1, Gamma_t, Xi_t, transmission_constraints])  # Shape (4 * U * F + U,)


In [67]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Lambda, Add
from tensorflow.keras.optimizers import Adam

In [68]:
import numpy as np
import random
from collections import deque

class PrioritizedReplayBuffer:
    def __init__(self, size, alpha=0.1):
        self.alpha = alpha  # Controls priority level (0 = uniform, 1 = fully prioritized)
        self.memory = deque(maxlen=size)
        self.priorities = deque(maxlen=size)

    def add(self, experience, error):
        """Adds an experience with priority."""
        priority = (abs(error) + 1e-5) ** self.alpha  # Avoid zero priority
        self.memory.append(experience)
        self.priorities.append(priority)

    def sample(self, batch_size, beta=0.4):
        """Samples experiences based on priority."""
        priorities = np.array(self.priorities)
        probs = priorities / priorities.sum()  # Normalize probabilities

        indices = np.random.choice(len(self.memory), batch_size, p=probs)
        experiences = [self.memory[idx] for idx in indices]

        # Compute importance sampling weights
        weights = (len(self.memory) * probs[indices]) ** (-beta)
        weights /= weights.max()  # Normalize weights

        return experiences, indices, np.array(weights, dtype=np.float32)

    def update_priorities(self, indices, errors):
        """Updates priorities after learning."""
        for idx, error in zip(indices, errors):
            self.priorities[idx] = (abs(error) + 1e-5) ** self.alpha

# Dueling DQN Model
def build_dueling_dqn(state_size, action_size):
    inputs = Input(shape=(state_size,))
    x = Dense(64, activation='relu')(inputs)
    x = Dense(64, activation='relu')(x)

    value = Dense(1, activation=None)(x)
    advantage = Dense(action_size, activation=None)(x)
    advantage = Lambda(lambda a: a - tf.reduce_mean(a, axis=1, keepdims=True))(advantage)

    q_values = Add()([value, advantage])
    model = Model(inputs, q_values)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

class D3QNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = PrioritizedReplayBuffer(size=10000, alpha=0.2)

        self.epsilon = 1.0  # Initial exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.998

        self.beta = 0.1  # Importance sampling weight for PER
        self.batch_size = 64

        self.model = build_dueling_dqn(state_size, action_size)
        self.target_model = build_dueling_dqn(state_size, action_size)
        self.target_model.set_weights(self.model.get_weights())

    def act(self, state):
        """Selects an action using epsilon-greedy policy with per-UAV F' constraint."""
        if np.random.rand() <= self.epsilon:
            # print("EXPLORATION\t******")
            action = np.zeros((U, F), dtype=int)

            for u in range(U):
                k = np.random.randint(0, F_prime + 1)  # Random number of files to cache
                if k > 0:
                    cached_files = np.random.choice(F, size=k, replace=False)
                    action[u, cached_files] = 1

            return action

        # print("EXPLOITATION\t******")
        q_values = self.model.predict(np.expand_dims(flatten_state(state), axis=0), verbose=0)

        # Reshape flat Q-values back into (U x F)
        q_matrix = q_values[0].reshape(U, F)
        action = np.zeros_like(q_matrix, dtype=int)

        for u in range(U):
            # Select top F' files based on Q-values for each UAV
            top_indices = np.argsort(q_matrix[u])[::-1][:F_prime]
            action[u, top_indices] = 1

        return action

    def remember(self, state, action, reward, next_state, error):
        """Stores experience in memory with priority."""
        self.memory.add((state, action, reward, next_state), error)

    def replay(self):
        """Samples experiences and trains the model."""
        if len(self.memory.memory) < self.batch_size:
            return

        experiences, indices, weights = self.memory.sample(self.batch_size, self.beta)
        states, actions, rewards, next_states = zip(*experiences)

        states = np.array([flatten_state(s) for s in states], dtype=np.float32)
        next_states = np.array([flatten_state(ns) for ns in next_states], dtype=np.float32)

        rewards = np.array(rewards)
        target_qs = rewards + 0.99 * np.max(self.target_model.predict(next_states, verbose=0), axis=1)

        q_values = self.model.predict(states, verbose=0)
        errors = np.abs(target_qs - np.max(q_values, axis=1))

        for i, action in enumerate(actions):
            q_values[i, action] = target_qs[i]

        self.model.fit(states, q_values, epochs=1, verbose=0, batch_size=self.batch_size)
        self.memory.update_priorities(indices, errors)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        self.beta = min(1.0, self.beta + 0.001)

    def update_target_model(self):
        """Updates the target model with new weights."""
        self.target_model.set_weights(self.model.get_weights())



In [ ]:
# Training
env = UAVCacheEnv(predicted_q_i_f, F_prime)
print(env.state_size,env.action_size)
agent = D3QNAgent(env.state_size, env.action_size)
episodes = 500
rewards = []
CHR = []
CHR_Max = []
Cost = []
SystemRegret = []
k=1


for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    H ,H_max,  system_regret , total_cost = 0,0,0,0
    # print(f"\nEpisode {episode+1}:")

    for step in range(10):  # Steps per episode
        action = agent.act(state)
        next_state, reward, H_ ,H_max_,  system_regret_ , total_cost_ , done , _ = env.step(action)
        # print(next_state, reward, done , _)
        q_values = agent.model.predict(np.expand_dims(flatten_state(next_state), axis=0), verbose=0)
        target = reward + 0.99 * np.max(agent.target_model.predict(np.expand_dims(flatten_state(next_state), axis=0), verbose=0))
        error = abs(target - np.max(q_values))
        # print("HELLLLLLLLLLLLLLLLO \t\t:", target,error)
        agent.remember(state, action, reward, next_state,error )
        agent.replay()
        # Print state, action, and reward at each step
        # print(f"Step {step+1}:")
        # print(f"  State: {state}")
        # print(f"  Action Taken: {action}")
        # print(f"  Reward: {reward:.4f}")
        state = next_state
        total_reward += reward
        # print(H, total_cost_)
        H = max(H, sum(H_)/len(H_))
        H_max = max(H_max, sum(H_max_)/len(H_max_))
        system_regret = max(system_regret, sum(system_regret_)/len(system_regret_))
        total_cost = max(total_cost, total_cost_)

    rewards.append(total_reward)
    CHR.append(H)
    CHR_Max.append(H_max)
    Cost.append(total_cost)
    SystemRegret.append(system_regret)
    agent.update_target_model()
    print(f"Episode {episode+1} completed. Total Reward: {total_reward:.2f}, Epsilon: {agent.epsilon:.4f}")


3208 800
Episode 1 completed. Total Reward: -71.89, Epsilon: 1.0000
Episode 2 completed. Total Reward: -73.61, Epsilon: 1.0000
Episode 3 completed. Total Reward: -75.32, Epsilon: 1.0000
Episode 4 completed. Total Reward: -73.28, Epsilon: 1.0000
Episode 5 completed. Total Reward: -73.77, Epsilon: 1.0000
Episode 6 completed. Total Reward: -74.57, Epsilon: 1.0000
Episode 7 completed. Total Reward: -72.02, Epsilon: 0.9861
Episode 8 completed. Total Reward: -72.95, Epsilon: 0.9665
Episode 9 completed. Total Reward: -76.08, Epsilon: 0.9474
Episode 10 completed. Total Reward: -75.23, Epsilon: 0.9286
Episode 11 completed. Total Reward: -66.71, Epsilon: 0.9102
Episode 12 completed. Total Reward: -68.63, Epsilon: 0.8922
Episode 13 completed. Total Reward: -69.12, Epsilon: 0.8745
Episode 14 completed. Total Reward: -66.15, Epsilon: 0.8571
Episode 15 completed. Total Reward: -72.66, Epsilon: 0.8402
Episode 16 completed. Total Reward: -57.85, Epsilon: 0.8235
Episode 17 completed. Total Reward: -55.

In [ ]:

# Display Final Cache Placement
optimal_cache = state
# print(len(optimal_cache))
print(f"\nOptimal Cache Placement: {optimal_cache['Gamma_t']}")

In [ ]:
action

In [ ]:

# Plot Reward Curve
plt.plot(np.array(CHR))
plt.plot(np.array(CHR_Max))
plt.plot(np.array(SystemRegret))
plt.plot(np.array(Cost)*cost_constant)
# plt.plot(np.array(rewards) + np.array(SystemRegret))
plt.xlabel("Episodes")
plt.ylabel("Reward")
plt.title("UAV Cache Placement - Reward Over Time")
plt.show()


In [ ]:
import json
import numpy as np

rewards_list = rewards

# Save to JSON 500_
with open("500iter_7F_16_random_X_rewards_per.json", "w") as f:
    json.dump(rewards_list, f)


In [ ]:
# Load from JSON
with open("500iter_7F_16_random_X_rewards_per.json", "r") as f:
    loaded_rewards = json.load(f)

# Plot again
import matplotlib.pyplot as plt

plt.plot(np.array(loaded_rewards))
plt.xlabel("Episodes")
plt.ylabel("Reward")
plt.title("UAV Cache Placement - Reward Over Time")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import json
import numpy as np

rewards_list = rewards

# Save to JSON file
with open("500iter_7F_16__Reward_rewards_per.json", "w") as f:
    json.dump(rewards_list, f)
with open("500iter_7F_16__CHR_rewards_per.json", "w") as f:
    json.dump(CHR, f)
with open("500iter_7F_16__CHRMAX_rewards_per.json", "w") as f:
    json.dump(CHR_Max, f)
with open("500iter_7F_16_Systemregret_rewards_per.json", "w") as f:
    json.dump(SystemRegret, f)
with open("500iter_7F_16_COST_rewards_per.json", "w") as f:
    json.dump(Cost, f)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json

predicted_hit_ratio = compute_cache_hit_ratio(action, state['predicted_Q_f'], UAVs_per_cluster)

actual_hit_ratio = compute_cache_hit_ratio(state['Xi_t'], actual_Q_f_all_uav, UAVs_per_cluster)

with open("500iter_7F_16_cache_hit_found_per_random_X.json", "w") as f:
    json.dump(predicted_hit_ratio.tolist(), f)

with open("500iter_7F_16_actual_cache_hit_found_per_random_X.json", "w") as f:
    json.dump(actual_hit_ratio.tolist(), f)

# X-axis: cluster index or UAV group
clusters = np.arange(1, len(predicted_hit_ratio) + 1)

# IEEE-style plot settings
plt.figure(figsize=(8, 5))
plt.plot(clusters, predicted_hit_ratio, marker='o', linestyle='-', color='blue', label='Predicted')
plt.plot(clusters, actual_hit_ratio, marker='s', linestyle='--', color='red', label='Actual')

# Styling for IEEE-like appearance
plt.grid(True, linestyle='--', alpha=0.6)
plt.xlabel('Cluster Index', fontsize=12)
plt.ylabel('Cache Hit Ratio', fontsize=12)
plt.title('Predicted vs Actual Cache Hit Ratio per Cluster', fontsize=14)
plt.xticks(clusters)
plt.ylim(0, 1)
plt.legend(loc='lower right', fontsize=10)
plt.tight_layout()

# Save or show
plt.savefig("500iter_7F_16_cache_hit_ratio_comparison_per.pdf", format='pdf')  # for IEEE paper
plt.show()


In [ ]:
count = 0
for j in range(0,len(state['Gamma_t'])):
    temp = 0
    for i in state['Gamma_t'][j]:
        if i&1:
            temp+=1
    print(temp)
print("F' = ", F_prime)

RANDOM CACHING


In [ ]:
import numpy as np
cache_matrix = np.zeros((U, F), dtype=int)

for u in range(U):
    cached_files = np.random.choice(F, size=F_prime, replace=False)
    cache_matrix[u, cached_files] = 1
print(cache_matrix)

count = 0
for j in range(0,len(cache_matrix)):
    temp = 0
    for i in cache_matrix[j]:
        if i&1:
            temp+=1
    print(temp)
print("F' = ", F_prime)


random_hit_ratio = compute_cache_hit_ratio(cache_matrix, actual_Q_f_all_uav, UAVs_per_cluster)
print(random_hit_ratio)

In [ ]:
print(sum(random_hit_ratio)/U)

In [ ]:

with open("500iter_7F_16_random_cache_hit_found_per_random_X.json", "w") as f:
    json.dump(random_hit_ratio.tolist(), f)
